# Using Zestful's free daily allowance to parse ingredients

In [1]:
import pandas as pd

In [2]:
posts = pd.read_csv("posts_preprocessed.csv")
posts_sample = pd.read_csv("posts_with_ingredients.csv")

posts['ingredient_lines'] = posts['ingredient_lines'].apply(eval)
posts_sample['ingredient_lines'] = posts_sample['ingredient_lines'].apply(eval)
posts_sample['ingredients_dict'] = posts_sample['ingredients_dict'].apply(eval)

In [42]:
# Already used posts 0,1,2,3,23,24,27,
# Time: 13:30
posts_new_sample = posts.iloc[[100,101,112]]

In [43]:
num_lines = 0
for lines in posts_new_sample['ingredient_lines']:
    print(f"Numer of lines {len(lines)}")
    num_lines += len(lines)
print(f"Total number of lines: {num_lines}")

Numer of lines 4
Numer of lines 11
Numer of lines 13
Total number of lines: 28


In [44]:
for lines in posts_new_sample['ingredient_lines']:
    print(lines)

['• 200 gram frozen banana', '• 100 gram frozen zucchini', '• 35 gram peanut flour', '• 200 gram silken tofu']
['1 pound chicken thighs or breast ', '1 garlic clove', '1 tablespoon butter', '1/3 cup honey ', '1/4 cup soy sauce ', '2 tablespoon rice vinegar ', '1 teaspoon chili flakes ', 'salt & pepper to taste', 'flour for dredging ', 'oil for frying ', 'scallions & parsley for garnish!']
['1 tablespoon olive oil', '1 leek, finely sliced', '1/2 fennel bulb, thinly sliced', '3 garlic clove, crushed', '1/2 teaspoon fennel seeds', '6 sprig oregano, or 1 1 /2 teaspoon dried leaves', '1 parmesan rind (such a genius way of adding flavour without the fat and calories of cheese)', '4 cup chicken or veg stock', '500 gram chicken breast, cut into large piece', '400 gram cannellini beans, drained', '1 zucchini, diced', '2 stick celery, chopped', '1/2 bunch silverbeet']


In [45]:
# zestful ingredient parser
import parse_ingredient

def get_ingredients_dict(ingredient_lines):
    
    ingredients = parse_ingredient.parse_multiple(ingredient_lines)
    
    return ingredients.as_dict()

In [46]:
posts_new_sample['ingredients_dict'] = posts_new_sample['ingredient_lines'].apply(get_ingredients_dict)

/var/folders/hn/dknbybgs0qv4myljwr8qtf4c0000gn/T/ipykernel_95980/3922682051.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  posts_new_sample['ingredients_dict'] = posts_new_sample['ingredient_lines'].apply(get_ingredients_dict)


In [47]:
for index, row in posts_new_sample.iterrows():
    print(row['ingredient_lines'])
    print(row['ingredients_dict'])
    print("\n-----------------------\n")

['• 200 gram frozen banana', '• 100 gram frozen zucchini', '• 35 gram peanut flour', '• 200 gram silken tofu']
[{'error': None, 'raw': '• 200 gram frozen banana', 'parsed': {'confidence': 0.8373106, 'product': 'frozen banana', 'productSizeModifier': None, 'quantity': 200.0, 'unit': 'gram', 'preparationNotes': None, 'usda_info': {'category': 'Fruits and Fruit Juices', 'description': 'Bananas, raw', 'fdcId': '173944', 'matchMethod': 'closestUnbranded'}}}, {'error': None, 'raw': '• 100 gram frozen zucchini', 'parsed': {'confidence': 0.471702, 'product': 'frozen zucchini', 'productSizeModifier': None, 'quantity': 100.0, 'unit': 'gram', 'preparationNotes': None, 'usda_info': None}}, {'error': None, 'raw': '• 35 gram peanut flour', 'parsed': {'confidence': 0.569126, 'product': 'peanut flour', 'productSizeModifier': None, 'quantity': 35.0, 'unit': 'gram', 'preparationNotes': None, 'usda_info': None}}, {'error': None, 'raw': '• 200 gram silken tofu', 'parsed': {'confidence': 0.273018, 'product

In [48]:
got_usda = 0
for ingredient_dict in posts_new_sample['ingredients_dict']:
    for ingredient in ingredient_dict:
        if ingredient['parsed']['usda_info']:
            got_usda += 1
print(f"Retrieved USDA info from {got_usda}/{num_lines} ingredients.")

Retrieved USDA info from 19/28 ingredients.


In [49]:
pd.concat([posts_sample,posts_new_sample]).to_csv("posts_with_ingredients.csv", index=False)

In [50]:
pd.read_csv("posts_with_ingredients.csv")

,Unnamed: 0,postUrl,profileUrl,username,fullName,commentCount,likeCount,pubDate,description,location,...,isSidecar,sidecarMedias,videoUrl,viewCount,language,score,description_preprocessed,includes_ingredient_list,ingredient_lines,ingredients_dict
0,81,https://www.instagram.com/p/Cgd_iR_vT2d/,https://www.instagram.com/all.about.oats,all.about.oats,Anushka Lodhi,0,2,2022-07-26T08:23:14.000Z,Chocolate fudge protein oatmeal🤎 💪\nIngredient...,"Ghaziabad, India",...,True,3.0,NaN,NaN,en,0.999997,chocolate fudge protein oatmeal \ningredients:...,True,"['1/2 cup instant oats', '1/2 cup milk', '1/2 ...","[{'error': None, 'raw': '1/2 cup instant oats'..."
1,104,https://www.instagram.com/p/Cgd8bLXDaCq/,https://www.instagram.com/a_m_eats,a_m_eats,Alice & Meg 🍴,3,12,2022-07-26T07:56:03.000Z,⁣Caprese Chicken with Pesto 🌿🍅⠀\n⠀\nThis dish ...,"Glasgow, United Kingdom",...,True,3.0,NaN,NaN,en,0.999997,⁣caprese chicken with pesto ⠀\n⠀\nthis dish is...,True,"['1 chicken breast⠀', '1 ball mozarella cheese...","[{'error': None, 'raw': '1 chicken breast⠀', '..."
2,169,https://www.instagram.com/p/CgdylqtOy9s/,https://www.instagram.com/hescottwellness,hescottwellness,"Natasha Hescott, RDN, CDN",0,1,2022-07-26T06:30:06.000Z,Looking for a fast recipe to make for lunch? T...,NaN,...,False,NaN,NaN,NaN,en,0.999994,looking for a fast recipe to make for lunch? t...,True,"['2 pound bok choy, small ', '1 tbs ginger, fr...","[{'error': None, 'raw': '2 pound bok choy, sma..."
3,172,https://www.instagram.com/p/CgdyENyLgmv/,https://www.instagram.com/rainbowpiatto,rainbowpiatto,Rainbow Piatto,1,14,2022-07-26T06:25:32.000Z,Sweet & Savoury Mushroom Scones~🍄 This easy an...,Singapore / Singapura / 新加坡 / சிங்கப்பூர்,...,False,NaN,NaN,NaN,en,0.999996,sweet & savoury mushroom scones~ this easy and...,True,"['200 gram swiss brown mushrooms, chopped ', '...","[{'error': None, 'raw': '200 gram swiss brown ..."
4,627,https://www.instagram.com/p/CgboCcgDiXs/,https://www.instagram.com/thealmostcook,thealmostcook,Mel 🍋,2,38,2022-07-25T10:19:26.000Z,overnight oats 🍯 if you’re wanting some easy a...,London Borough of Hackney,...,False,NaN,NaN,NaN,en,0.999996,overnight oats if you’re wanting some easy and...,True,"['- 1.5 cup oats', '- 2 scoops unflavoured or ...","[{'error': None, 'raw': '- 1.5 cup oats', 'par..."
5,635,https://www.instagram.com/p/Cgbk1gkj0Bj/,https://www.instagram.com/hangryinlondon,hangryinlondon,Hangry in London 🌶,0,16,2022-07-25T09:51:27.000Z,Finally got round to making baba ganoush! Not ...,Cornwall,...,True,2.0,NaN,NaN,en,0.999998,finally got round to making baba ganoush! not ...,True,"['2 aubergines ', '2 minced garlic clove ', '2...","[{'error': None, 'raw': '2 aubergines ', 'pars..."
6,781,https://www.instagram.com/p/Cgdw3BDKgw2/,https://www.instagram.com/ket_ofoodrecipesandtips,ket_ofoodrecipesandtips,Keto Food Recipes and Tips,1,5,2022-07-26T06:18:07.000Z,👉Save this post for the recipe.\nFollow \n@ket...,NaN,...,False,NaN,https://scontent-lhr8-2.cdninstagram.com/v/t50...,6.0,en,0.999996,save this post for the recipe.\nfollow \n to ...,True,"['1.5 cup iceberg lettuce', '3 tablespoon mayo...","[{'error': None, 'raw': '1.5 cup iceberg lettu..."
7,564,https://www.instagram.com/p/ChCwo1JoMzj/,https://www.instagram.com/dtmdiaries,dtmdiaries,Duaa Tariq | Content Creator,5,3,2022-08-09T15:04:58.000Z,Recipe time!!\n\nBoil your lasagna for 5-7 min...,NaN,...,True,4.0,NaN,NaN,en,0.999995,recipe time!!\n\nboil your lasagna for 5-7 min...,True,"['- half kg chicken minced', '- 1 medium sized...","[{'error': None, 'raw': '- half kg chicken min..."
8,599,https://www.instagram.com/p/ChCujbgpwVP/,https://www.instagram.com/the_cook_book_25,the_cook_book_25,Foodie❤,0,9,2022-08-09T14:46:45.000Z,Banana smoothie🤍\n. \n. \n♡Ingredients:\n1 gla...,NaN,...,False,NaN,NaN,NaN,en,0.999997,banana smoothie\n. \n. \n♡ingredients:\n1 glas...,True,"['1 glass milk', '2 banana', '1/4 cup oats', '...","[{'error': None, 'raw': '1 glass milk', 'parse..."
9,704,https://www.instagram.com/p/Ch